# Network Complexity

This is an indicator for the 2023 base blueprint re-run.

Created by Amy Keister, last run by Amy Keister on 23 June 2023. It took about 4 hours to run

In [1]:
import os
import arcpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\NetworkComplexity\Networ.gdb"

In [4]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\NetworkComplexity\NetworkComplexity.tif"

In [5]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [6]:
# define inputs
# functional network lines from Brendan by region
FNL02= r"F:\GIS_DATA\WaterResources\SARP2023\SARP_dams_networks_3_13_2023\region02_dams_networks.gdb\region02_dams_networks"
FNL03= r"F:\GIS_DATA\WaterResources\SARP2023\SARP_dams_networks_3_13_2023\region03_dams_networks.gdb\region03_dams_networks"
FNL05= r"F:\GIS_DATA\WaterResources\SARP2023\SARP_dams_networks_3_13_2023\region05_dams_networks.gdb\region05_dams_networks"
FNL06= r"F:\GIS_DATA\WaterResources\SARP2023\SARP_dams_networks_3_13_2023\region06_dams_networks.gdb\region06_dams_networks"
FNL07= r"F:\GIS_DATA\WaterResources\SARP2023\SARP_dams_networks_3_13_2023\region07_dams_networks.gdb\region07_dams_networks"
FNL08= r"F:\GIS_DATA\WaterResources\SARP2023\SARP_dams_networks_3_13_2023\region08_dams_networks.gdb\region08_dams_networks"
FNL10= r"F:\GIS_DATA\WaterResources\SARP2023\SARP_dams_networks_3_13_2023\region10_dams_networks.gdb\region10_dams_networks"
FNL11= r"F:\GIS_DATA\WaterResources\SARP2023\SARP_dams_networks_3_13_2023\region11_dams_networks.gdb\region11_dams_networks"
FNL12= r"F:\GIS_DATA\WaterResources\SARP2023\SARP_dams_networks_3_13_2023\region12_dams_networks.gdb\region12_dams_networks"
FNL13= r"F:\GIS_DATA\WaterResources\SARP2023\SARP_dams_networks_3_13_2023\region13_dams_networks.gdb\region13_dams_networks"
FNL21= r"F:\GIS_DATA\WaterResources\SARP2023\SARP_dams_networks_3_13_2023\region21_dams_networks.gdb\region21_dams_networks"
# epa floodplain
FP= r"F:\GIS_DATA\WaterResources\Estimated_floodplain_CONUS\Estimated_floodplain_CONUS.tif"
# I'm going to test my code on the national download of HR catchments, but I think I need to download indivdually from the secret server
catch= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\NHDPlus_H_National_Release_1_GDB.gdb\NHDPlusCatchment"
# these are the medium resolution catchments that don't have the nearshore problems in NC
Mcatch= r"F:\GIS_DATA\WaterResources\NHD\NHDPlusV21\NHDPlusV21_Catchment.gdb\CatchmentSECAS"

In [7]:
# define inputs
# FEMA floodplain
#FP = r"F:\GIS_DATA\WaterResources\Estimated_floodplain_CONUS\Estimated_floodplain_CONUS.tif"
# SARP funtional network lines
#FNL= r"F:\GIS_DATA\WaterResources\NHD\VIPR_FromBrendan\region21_dams_networks_1_24_2023\region21_dams_networks.gdb\region21_dams_networks"
# SARP functional network line segments
#FNLS= r"F:\GIS_DATA\WaterResources\NHD\VIPR_FromBrendan\region21_dams_networks_1_24_2023\region21_dams_segments.gdb\region21_dams_segments"
# NHDPLusHR catchments from national download
#catch= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\NHDPlus_H_National_Release_1_GDB.gdb\NHDPlusCatchment"
# NHDPlusHR flowlines from national download
#flow= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\NHDPlus_H_National_Release_1_GDB.gdb\NetworkNHDFlowline"


# NHDPlus Medium Resolution Catchments
#catch = r"F:\GIS_DATA\WaterResources\NHD\NHDPlusV21\NHDPlusV21_Catchment.gdb\Catchment"
# can't use high resolution for this purpose because the NC coast is messed up
#catch= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\NHDPlus_H_National_Release_1_GDB.gdb\NHDPlusCatchment"

### Start Analysis

In [8]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [9]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\NetworkComplexity\Networ.gdb


### Bring in SARP high res data

In [11]:
# merge functional network lines (dams networds) together
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.Merge([FNL02,FNL03,FNL05,FNL06,FNL07,FNL08,FNL10,FNL11,FNL12,FNL13,FNL21], "FNL")
    #arcpy.management.Merge([FNL02,FNL05], "FNLtest")
    #arcpy.management.Merge("region02_dams_networks;region03_dams_networks", r"D:\ArcProProjects\BlankWLinks2\Default.gdb\region02_dams_networks_Merge", 'sizeclasses "sizeclasses" true true false 2 Short 0 0,First,#,region02_dams_networks,sizeclasses,-1,-1,region03_dams_networks,sizeclasses,-1,-1', "NO_SOURCE_INFO")

In [12]:
# convert Brendan's high res linework to raster, using the "sizes" field, which represents network complexity
# before I calaculated this at the original NHDPlus raster cell size, but I don't think that matters too much
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, mask=Rextent):
    arcpy.conversion.FeatureToRaster(in_features="FNL", field="sizeclasses", out_raster="FNLsizeclss", cell_size=Rextent)

In [13]:
# the steps above should have clipped the data down to the extent, but it didn't so I'm going to do that now
with arcpy.EnvManager(snapRaster=Rextent, extent=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Times(Rextent, "FNLsizeclss"); out_raster.save("FNLsizeclss2")

In [14]:
# use zonal statistics to assign "sizes" value from Brendan's linework to the national NHDPlus HR catchments
# we need to do this to turn the linework into an area. The network complexity is calculated by Brendan
# using line network analysis, for the blueprint, we need to assign those lines back to an area, we start by
# assigning them back to the catchment that intersects with the lines.
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ZonalStatistics(catch, "OBJECTID", "FNLsizeclss2", "MAJORITY", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZBcatSizesMaj")

### Prep data to make a zero class
zeros will be areas where there are NHD data, and NoData will be areas where there is not NHD data

In [15]:
# make a copy of catchment layer for edits
with arcpy.EnvManager(snapRaster=Rextent, extent=Rextent, cellSize=Rextent):
    arcpy.management.CopyFeatures(catch, "catch", '', None, None, None)

In [16]:
# add field to the catchment layer to prepare to convert to raster
arcpy.management.CalculateField("catch", "raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\NetworkComplexity\\Networ.gdb\\catch'>

In [17]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster("catch", "raster", "catch1", 30)

### Clip the zonal statistics layer to floodplain and buffered flow lines to make a preliminary network complexity layer

In [18]:
# use conditional statement to limit results to the floodplain layer
# the result is the prelimiary network complexity layer
# this layer has "holes" in it where Brendan has had to remove the "loops" of braided streams
# so he can do the network analysis
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con(FP, "ZBcatSizesMaj", "", where_clause="Value = 1"); out_raster.save("prelimNC")

### Deal with missing catchments inside floodplain

because Brendan has to remove loops in the network some catchments inside the floodplain do not intersect with the linework that Brendan has calculated network complexity on. These should be at least a value of 1, because they are in the floodplain. Some should have values higher than that. Normally these are small catchments in braided stream networks in large floodplains. They may be separated from the floodplain by a natural levee, but during flood stage, they should be connected to the flood plain around them.

In [19]:
# give all areas that didn't intersect with Brendan's data a crazy value we can keep track of
out_raster = arcpy.sa.Reclassify("prelimNC", "Value", "NODATA 20", "DATA"); out_raster.save("prelimNC20")

In [20]:
# take catchment layer times floodplain layer to help find missing catchments that don't intersect 
# Brendan's linework. We don't want to map a floodplain anywhere there 
# isn't also NHD catchments
out_raster= arcpy.sa.Times("catch1", FP); out_raster.save("FPcatch")

In [21]:
# clip back down to floodplain/NHD catchment area
# areas with the crazy value in this raster are catchments that should be at least a value of 1
out_raster= arcpy.sa.Times("prelimNC20", "FPcatch"); out_raster.save("prelimNC20clip")

In [22]:
# pull those areas that are in the floodplain, that have NHD catchment data, and that do not intersect 
# brendan linework (crazy value) as a unique raster
out_raster = arcpy.sa.Con("prelimNC20clip", "1", "", where_clause="Value = 20"); out_raster.save("missing")

In [23]:
# expand by one pixel
out_raster = arcpy.sa.Expand("missing", 1, [1], ""); out_raster.save("missingExpnd1")

In [24]:
# use a region group to identify individual missing areas
out_raster = arcpy.sa.RegionGroup("missingExpnd1", "FOUR", "WITHIN", "ADD_LINK", None); out_raster.save("missingExpnd1rg")

In [25]:
# calculate maximum value in the preliminary network complexity that intsersect expanded missing areas
# useing zonal statistics
out_raster = arcpy.sa.ZonalStatistics("missingExpnd1rg", "Value", "prelimNC", "MAXIMUM", "DATA", "CURRENT_SLICE", 90); out_raster.save("MissingZstatPncMax")

In [26]:
# use a conditional statement to assign maximum nearby values to the missing areas in the preliminary network complexity
out_raster = arcpy.sa.Con("prelimNC20clip", "MissingZstatPncMax", "prelimNC20clip", where_clause="Value = 20"); out_raster.save("prelimNCfill")

In [27]:
# reclassify to make NoData zero
out_raster = arcpy.sa.Reclassify("prelimNCfill", "Value", "NODATA 0;1 1;2 2;3 3;4 4", "DATA"); out_raster.save("prelimNCfill0")

In [28]:
# clip back down to NHD catchment area
out_raster= arcpy.sa.Times("prelimNCfill0", "catch1"); out_raster.save("ind")

### Explore fixing incorrect 1 values inside major floodplains

Similar issue to the missing catchments, we also have catchments with artificially low values inside major floodplains.
They may be separated from the floodplain by a natural levee, but during flood stage, they should be connected to the flood plain around them.
Fixing them is trickier, as we may accidentally "fix" something that actually wasn't an error.

### Clip down to extent of medium resolution catchments 
The high res catchments have issues in the coast

In [29]:
# make a copy of catchment layer for edits
with arcpy.EnvManager(snapRaster=Rextent, extent=Rextent, cellSize=Rextent):
    arcpy.management.CopyFeatures(Mcatch, "Mcatch", '', None, None, None)

In [30]:
# add field to the catchment layer to prepare to convert to raster
arcpy.management.CalculateField("Mcatch", "raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\NetworkComplexity\\Networ.gdb\\Mcatch'>

In [31]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster("Mcatch", "raster", "Mcatch1", 30)

In [32]:
# clip down to Medium resolution NHD catchment area
out_raster= arcpy.sa.Times("ind", "Mcatch1"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [35]:
# clip to extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("Mask")

In [36]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("Mask", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [33]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 7:
        return '7 = 7 connected stream classes'
    elif value == 6:
        return '6 = 6 connected stream classes'
    elif value == 5:
        return '5 = 5 connected stream classes'
    elif value == 4:
        return '4 = 4 connected stream classes'
    elif value == 3:
        return '3 = 3 connected stream classes'
    elif value == 2:
        return '2 = 2 connected stream classes'
    elif value == 1:
        return '1 = 1 connected stream class'
    elif value == 0:
        return '0 = Not identified as a floodplain'
"""
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\NetworkComplexity\\Networ.gdb\\indicator'>

In [13]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 7:
		return 1
	if Value == 6:
		return 1
	if Value == 5:
		return 7
	if Value == 4:
		return 60
	if Value == 3:
		return 114
	if Value == 2:
		return 175
	if Value == 1:
		return 215
	else:
		return 255
		
def Reclass2(Value):
	if Value == 7:
		return 70
	if Value == 6:
		return 109
	if Value == 5:
		return 130
	if Value == 4:
		return 147
	if Value == 3:
		return 172
	if Value == 2:
		return 193
	if Value == 1:
		return 213
	else:
		return 255
		
def Reclass3(Value):
	if Value == 7:
		return 54
	if Value == 6:
		return 91
	if Value == 5:
		return 143
	if Value == 4:
		return 194
	if Value == 3:
		return 209
	if Value == 2:
		return 221
	if Value == 1:
		return 233
	else:
		return 255
"""
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\NetworkComplexity\\NetworkComplexity.tif'>

In [37]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

13600.865392684937
